In [1]:
# import importlib
# importlib.reload(cluster_util)
# import cluster_util

In [ ]:
import os
import sys

sys.path.insert(1, os.path.realpath(os.path.pardir))

# Notebook error analysis 

## Evaluate ml linker

In [5]:
import pandas as pd
import config

df_err_k = pd.read_excel(config.path_kaggle_error_process.joinpath('nberror_k_eid_p.xlsx'))
df_err_g = pd.read_excel(config.path_github_error_process.joinpath('nberror_g_all_eid_p.xlsx'))

df_err_k = df_err_k[df_err_k.is_MLnb&df_err_k.is_relevant&df_err_k.if_ast==1.0]
df_err_g = df_err_g[df_err_g.is_MLnb&df_err_g.is_relevant&df_err_g.if_ast==1.0]

In [8]:
print(df_err_k.fname.nunique())
print(df_err_g.fname.nunique())

2706
66771


In [18]:
# sample on fname/notebooks

import numpy as np

n = 30
k_sampled_fname = np.random.choice(df_err_k.fname.unique(), n, replace=False)
print(len(np.unique(k_sampled_fname)))

g_sampled_fname = np.random.choice(df_err_g.fname.unique(), n, replace=False)
print(len(np.unique(g_sampled_fname)))

30
30


In [36]:
# then further sample on eid/1 error per notebook

k_sampled_fname_eid = (df_err_k[df_err_k.fname.isin(k_sampled_fname)]).groupby('fname').apply(
    pd.DataFrame.sample, 
    n=1,
    include_groups=True
).reset_index(level=0, drop=True).sort_index()

g_sampled_fname_eid = (df_err_g[df_err_g.fname.isin(g_sampled_fname)]).groupby('fname').apply(
    pd.DataFrame.sample, 
    n=1,
    include_groups=True
).reset_index(level=0, drop=True).sort_index()

C:\Users\yirwa29\AppData\Local\Temp\ipykernel_3196\889860243.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  k_sampled_fname_eid = (df_err_k[df_err_k.fname.isin(k_sampled_fname)]).groupby('fname').apply(
C:\Users\yirwa29\AppData\Local\Temp\ipykernel_3196\889860243.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  g_sampled_fname_eid = (df_err_g[df_err_g.fname.isin(g_sampled_fname)]).groupby('fname').a

In [43]:
k_sampled_fname_eid.to_excel(config.path_kaggle_error_process.joinpath("nberror_k_eid_p_mllinker_validation_sample0.xlsx"),
                             index=False, engine='xlsxwriter')
g_sampled_fname_eid.to_excel(config.path_github_error_process.joinpath("nberror_g_all_eid_p_mllinker_validation_sample0.xlsx"),
                             index=False, engine='xlsxwriter')